In [81]:
import requests
import pandas as pd
from datetime import datetime

# URL da API Prometheus
url = 'http://192.168.242.131:9090/api/v1/query?query=container_cpu_usage_seconds_total{pod=~".*postgres.*"}[10m]'

# Fazendo a requisição GET
response = requests.get(url)
data = response.json()  # Converte a resposta para dict

# Verifica se há resultados e estrutura esperada
results = data.get('data', {}).get('result', [])

# Lista para armazenar os registros
records = []

# Processa cada resultado
for item in results:
    # Extrai a série temporal (samples)
    for value in item.get('values', []):
        ts = float(value[0])  # Timestamp UNIX
        cpu_val = float(value[1])
        # Converte timestamp para Data/Hora legível
        dt = datetime.utcfromtimestamp(ts)
        
        # Extrai o nome do pod dos labels
        pod_name = item['metric'].get('pod', 'N/A')
        records.append({"timestamp": dt, "pod": pod_name, "cpu": cpu_val})

# Cria DataFrame
df = pd.DataFrame(records)
#df = df.sort_values(by='timestamp', ascending=False)
# Mostra resultado
df

,timestamp,pod,cpu
0,2025-09-04 16:27:43.796,postgres-deployment-76d6f4778d-ccs8g,45105.035789
1,2025-09-04 16:27:58.720,postgres-deployment-76d6f4778d-ccs8g,45105.039263
2,2025-09-04 16:28:11.293,postgres-deployment-76d6f4778d-ccs8g,45105.039409
3,2025-09-04 16:28:27.516,postgres-deployment-76d6f4778d-ccs8g,45105.039500
4,2025-09-04 16:28:43.066,postgres-deployment-76d6f4778d-ccs8g,45105.039667
...,...,...,...
102,2025-09-04 16:35:50.493,postgres-deployment-76d6f4778d-ccs8g,0.023826
103,2025-09-04 16:36:08.881,postgres-deployment-76d6f4778d-ccs8g,0.023826
104,2025-09-04 16:36:33.592,postgres-deployment-76d6f4778d-ccs8g,0.023826
105,2025-09-04 16:36:53.398,postgres-deployment-76d6f4778d-ccs8g,0.023826
